<a href="https://colab.research.google.com/github/d112358/idlmav/blob/main/environments/explore_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kaggle environment notes and experiments
This notebook contains several experiments performed on Kaggle

## Running library notebooks on Kaggle
TODO

### Notebook links
* [00_overview.ipynb](../nbs/00_overview.ipynb)
* [01_explore_layout.ipynb](../nbs/01_explore_layout.ipynb)
* [02_test_layout.ipynb](../nbs/02_test_layout.ipynb)
* [03_explore_rendering.ipynb](../nbs/03_explore_rendering.ipynb)
* [04_explore_plotly_interaction.ipynb](../nbs/04_explore_plotly_interaction.ipynb)
* [05_explore_js_interaction.ipynb](../nbs/05_explore_js_interaction.ipynb)
* [06_test_rendering.ipynb](../nbs/06_test_rendering.ipynb)
* [07_explore_tracing.ipynb](../nbs/07_explore_tracing.ipynb)
* [08_fx_profiling_tutorial.ipynb](../nbs/08_fx_profiling_tutorial.ipynb)
* [09_explore_fx_interpreter.ipynb](../nbs/09_explore_fx_interpreter.ipynb)
* [10_test_tracing.ipynb](../nbs/10_test_tracing.ipynb)
* [11_explore_colors.ipynb](../nbs/11_explore_colors.ipynb)
* [12_test_idlmav.ipynb](../nbs/12_test_idlmav.ipynb)



### Steps
* Open the notebook from the links below
* Paste the following code above the first cell
  ```python
  from google.colab import output
  output.enable_custom_widget_manager()
  %pip install git+https://github.com/d112358/idlmav.git
  %pip install torchview
  %pip install torchinfo
  %pip install colorspacious
  ```
* Run the notebook normally



### Notes
* In general, wherever the `plotly_renderer_context` context manager is used, pass `'kaggle'` as its argument for better results on Kaggle

## Information about existing environment

In [ ]:
import sys, os, platform
from datetime import datetime
from pathlib import Path

### OS, Kaggle Kernel and Python

In [ ]:
print(f"Operating System: {platform.system()} {platform.release()} ({platform.version()})")
!cat /etc/os-release
print("")
print(f"Kaggle Kernel Version: {os.getenv('KAGGLE_KERNEL_VERSION')}")
print(f"Python Version: {sys.version}")
print(f"Current Date and Time: {datetime.now()}")

Operating System: Linux 6.1.85+ (#1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024)
PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy
Current Date and Time: 2024-12-10 12:28:41.040460
Python Version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]


### Existing packages and versions

In [ ]:
import subprocess
filter_strings = ["beautifulsoup", "colorspacious", "fast", "graphviz", "ipy", "jsbeautifier", "matplotlib", "munkres", "nbformat", "numpy", "pandas", "plotly", "tabulate", "timm", "torch"]
result = subprocess.run(["pip", "list"], stdout=subprocess.PIPE, text=True)
lines = result.stdout.split("\n")

filtered_lines = [line for line in lines if any(s in line for s in filter_strings)]
print("\n".join(filtered_lines))

beautifulsoup4                     4.12.3
fastai                             2.7.18
fastcore                           1.7.22
fastdownload                       0.0.7
fastjsonschema                     2.21.1
fastprogress                       1.0.3
fastrlock                          0.8.2
geopandas                          1.0.1
graphviz                           0.20.3
ipyevents                          2.0.2
ipyfilechooser                     0.6.0
ipykernel                          5.5.6
ipyleaflet                         0.19.2
ipyparallel                        8.8.0
ipython                            7.34.0
ipython-genutils                   0.2.0
ipython-sql                        0.5.0
ipytree                            0.2.2
ipywidgets                         7.7.1
matplotlib                         3.8.0
matplotlib-inline                  0.1.7
matplotlib-venn                    1.1.1
nbformat                           5.10.4
numpy                              1.26.4
pandas 

### Memory and disk space

In [ ]:
import psutil, shutil
print(f"Processor: {platform.processor()}")
print(f"CPU Count: {psutil.cpu_count(logical=True)}")
print(f"Total RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")
print(f"Disk Space: {shutil.disk_usage('/').total / 1e9:.2f} GB")

In [ ]:
print("Memory Usage:")
!free -h
print("")
print("Disk Space:")
!df -h

Memory Usage:
               total        used        free      shared  buff/cache   available
Mem:            12Gi       860Mi       8.6Gi       1.0Mi       3.2Gi        11Gi
Swap:             0B          0B          0B

Disk Space:
Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   33G   76G  31% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.2G  820M  59% /usr/sbin/docker-init
tmpfs           6.4G  116K  6.4G   1% /var/colab
/dev/sda1        50G   34G   17G  68% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


### Internet connectivity

In [ ]:
import requests
try:
    response = requests.get("https://www.google.com", timeout=5)
    print(f"Internet Access: OK (Status Code {response.status_code})")
except requests.ConnectionError:
    print("Internet Access: Failed")

### Git head revision

In [ ]:
!git rev-parse HEAD

78779e076d66a0e0d31c5f94c2fcc0b55a6a754f


## Installation experiments

### Confirm we're running on Colab

In [1]:
import sys, os, platform, warnings
from datetime import datetime
from pathlib import Path

try:
    import google.colab
    print('Running in Colab')
except ImportError:
    warnings.warn('This notebook is designed to be executed in Colab. You are not currently running in Colab')

class StopExecution(Exception):
    def _render_traceback_(self):
        return []

def check_colab():
    try:
        import google.colab
    except ImportError:
        print('Skipping cell. This notebook is designed to be executed in Colab')
        raise StopExecution

Running in Colab


### Install IDLMAV

In [2]:
check_colab()
%pip install git+https://github.com/d112358/idlmav.git

  Cloning https://github.com/d112358/idlmav.git to /tmp/pip-req-build-b5p0ns9n
  Running command git clone --filter=blob:none --quiet https://github.com/d112358/idlmav.git /tmp/pip-req-build-b5p0ns9n
  Resolved https://github.com/d112358/idlmav.git to commit acc03f70a25b22ae797e08b0ff3b0fe2daa58ba1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for idlmav: filename=idlmav-1.0.0-py3-none-any.whl size=34853 sha256=3b062b806a41af722a991164635ab650c78be175a718566b6b178c03bd21f72f
  Stored in directory: /tmp/pip-ephem-wheel-cache-_m9vytnn/wheels/51/a4/e6/3bc66208df7c6cbe818b92bf4fc2f55aff92c4b31b8e2bbf5a
Successfully built idlmav


### Additional dependencies used in development and testing
* `timm` and `fastcore` are required for some models with which this library is tested
* `torchview` and `torchinfo` are used to verify the correctness of the model tracing algoritm
  - `graphviz` is required by `torchview`
* `colorspacious` was used to convert RGB colors to CIELAB to select the colors to used for nodes

In [4]:
check_colab()
%pip install timm
%pip install fastcore
%pip install torchview
%pip install torchinfo
%pip install graphviz
%pip install colorspacious

### Install miniai (optional)
* See notes in [setup_vscode_wsl.ipynb](./setup_vscode_wsl.ipynb)
* Some of the notebooks in this library perform experiments on models built with the `miniai` library from the [2022 fastai course](https://course.fast.ai/)
* The installation of `datasets` sometimes fails on Colab due to 2nd order dependency version conflicts, but so far this has not broken the functionality we're interested in
  - `miniai` imports `datasets`, but we're not using features from `miniai` that uses `datasets` intensively

In [6]:
# Archive the miniai repo to the `tmp` directory and extract the archive to the `miniai` directory
check_colab()
from pathlib import Path
Path('tmp').mkdir(exist_ok=True)     # Location to archive the repo to
Path('miniai').mkdir(exist_ok=True)  # Location to extract the archive to
!git -C tmp init -b master
!git -C tmp remote add origin https://github.com/fastai/course22p2.git
!git -C tmp fetch --depth=1 origin master
!git -C tmp archive --format=tar origin/master:miniai | tar -x -C miniai
!rm -rf tmp

Initialized empty Git repository in /content/tmp/.git/
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 145 (delta 10), reused 82 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (145/145), 28.36 MiB | 1.01 MiB/s, done.
Resolving deltas: 100% (10/10), done.
From https://github.com/fastai/course22p2
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master


In [7]:
# Install miniai dependencies
check_colab()
%pip install fastprogress
%pip install torcheval
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### Test imports

In [8]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from idlmav import MAV
from miniai.init import clean_mem

## Plotly experiments

In [10]:
import plotly.io as pio
import plotly.graph_objects as go

data = {
    'name': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T'],
    'x': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'y': [10, 15, 13, 17, 11, 13, 18, 14, 12, 19, 17, 15, 10, 19, 18, 13, 12, 17, 14, 15]
}

### Available renderers

In [9]:
available_renderers = list(pio.renderers)
print(f'Available renderers: {", ".join(available_renderers)}')

Available renderers: plotly_mimetype, jupyterlab, nteract, vscode, notebook, notebook_connected, kaggle, azure, colab, cocalc, databricks, json, png, jpeg, jpg, svg, pdf, browser, firefox, chrome, chromium, iframe, iframe_connected, sphinx_gallery, sphinx_gallery_png


### Standard figure

In [14]:
fig = go.Figure(
    go.Scatter(x=data['x'], y=data['y'], mode='markers',
               hovertemplate=('name: %{customdata}'),
               customdata=data['name']
    )
)
fig.show()


### Figure widget

In [15]:
from google.colab import output
output.enable_custom_widget_manager()

widget = go.FigureWidget(
    data=go.Scatter(x=data['x'], y=data['y'], mode='markers',
        hovertemplate=('name: %{customdata}'),
        customdata=data['name']
    )
)

display(fig)

### Figure widget inside ipywidgets container

In [18]:
import ipywidgets as widgets
from google.colab import output
output.enable_custom_widget_manager()

widget = go.FigureWidget(
    data=go.Scatter(x=data['x'], y=data['y'], mode='markers',
        hovertemplate=('name: %{customdata}'),
        customdata=data['name']
    )
)

min_y, max_y = min(data['y']), max(data['y'])
slider = widgets.FloatRangeSlider(
    value=[min_y, max_y], min=min_y, max=max_y, step=0.01,
    description='Y Range:', orientation='vertical', continuous_update=True,
    layout=widgets.Layout(height='400px'))

container1 = widgets.HBox([slider, widget])

def update_plot(change):
    widget.update_layout(yaxis=dict(range=slider.value))

slider.observe(update_plot, names="value")
display(container1)

## IDLMAV experiments

In [19]:
import torch, torchvision
from idlmav import MAV, plotly_renderer_context
from IPython.display import display
device = 'cpu'
model = torchvision.models.resnet18().to(device)
x = torch.randn(16,3,160,160).to(device)
mav = MAV(model, x, device=device)
container2 = mav.draw_interactive_graph(add_slider=True, add_overview=False, add_table=False)
display(container2)

In [20]:
from idlmav import InteractiveViewer
v = InteractiveViewer(mav.tracer.g)
v.draw(add_slider=True, add_overview=False, add_table=False)

In [38]:
# Setup parameters
num_levels_displayed = 20
height_px = 400
g = v.g
initial_y_range = v.fit_range([v.in_level+num_levels_displayed-0.5, v.in_level-0.5], v.full_y_range)
initial_x_range = v.full_x_range

# Create the main panel
main_panel_layout = widgets.Layout(flex = '0 1 auto', margin='0px', padding='0px', overflow='hidden')
main_fig_layout = go.Layout(
    width=max((v.graph_num_cols*100, 180)), height=height_px,
    plot_bgcolor='#e5ecf6',
    autosize=True,
    xaxis=dict(range=initial_x_range, showgrid=False, zeroline=False, visible=False),
    yaxis=dict(range=initial_y_range, showgrid=False, zeroline=False, visible=False),
    margin=dict(l=0, r=2, t=1, b=1),
    showlegend=False,
    title=dict(text=None)
)
v.main_fig = go.FigureWidget(layout=main_fig_layout)
v.main_panel = widgets.Box(children=[v.main_fig], layout=main_panel_layout)
panels = [v.main_panel]

# Add connections lines between the nodes
line_traces = []
for ci,c in enumerate(g.connections):
    if ci>20: break
    x_coords, y_coords = v.get_connection_coords(c)
    line_trace = go.Scatter(
        x=x_coords, y=y_coords, mode="lines",
        line=dict(color="gray", width=1),
        showlegend=False
    )
    v.main_fig.add_trace(line_trace)
    line_traces.append(line_trace)

# Add the node markers
node_trace = go.Scatter(
    x=[n.x for n in g.nodes],
    y=[n.y for n in g.nodes],
    mode='markers',
    marker=dict(
        size=[v.params_to_dot_size(n.params) for n in g.nodes],
        color=[v.get_node_color(n) for n in g.nodes],
        colorscale='Bluered'
    ),
    hovertemplate=(
        'Name: %{customdata[0]}<br>' +
        'Operation: %{customdata[1]}<br>' +
        'Activations: %{customdata[2]}<br>' +
        'Parameters: %{customdata[3]}<br>' +
        'FLOPS: %{customdata[4]}<br>' +
        '<extra></extra>'
    ),
    customdata=[v.node_data(n) for n in g.nodes],
    showlegend=False
)
v.main_fig.add_trace(node_trace)
node_trace_idx = len(v.main_fig.data)-1

# Add slider
# * Use negative values everywhere, because ipywidgets does not support
#   inverting the direction of vertical sliders
slider_panel_layout = widgets.Layout(flex = '0 0 auto', margin='0px', padding='0px', overflow='visible')
v.slider_widget = widgets.FloatRangeSlider(
    value=[-initial_y_range[0], -initial_y_range[1]], min=-v.full_y_range[0], max=-v.full_y_range[1],
    step=0.01, description='', orientation='vertical', continuous_update=True,
    layout=widgets.Layout(height=f'{height_px}px')
)
v.slider_widget.readout = False  # For some reason it does not seem to work if set during construction
v.slider_panel = widgets.Box(children=[v.slider_widget], layout=slider_panel_layout)
panels.insert(0, v.slider_panel)

# Create container for all panels
# * To be displayed in Notebook using `display`
container_layout = widgets.Layout(
    width='100%',
    margin='0px', padding='0px')
container = widgets.HBox(panels, layout=container_layout)

# Set up event handlers
v.main_fig.data[node_trace_idx].on_click(v.on_main_panel_click)
v.main_fig.layout.on_change(v.on_main_panel_pan_zoom, 'xaxis.range', 'yaxis.range')
v.slider_widget.observe(v.on_slider_value_change, names="value")

# Restrict actions on plots
# v.main_fig.update_layout(config=dict(displayModeBar=False))
# [ "autoScale2d", "autoscale", "editInChartStudio", "editinchartstudio", "hoverCompareCartesian", "hovercompare", "lasso", "lasso2d", "orbitRotation", "orbitrotation", "pan", "pan2d", "pan3d", "reset", "resetCameraDefault3d", "resetCameraLastSave3d", "resetGeo", "resetSankeyGroup", "resetScale2d", "resetViewMap", "resetViewMapbox", "resetViews", "resetcameradefault", "resetcameralastsave", "resetsankeygroup", "resetscale", "resetview", "resetviews", "select", "select2d", "sendDataToCloud", "senddatatocloud", "tableRotation", "tablerotation", "toImage", "toggleHover", "toggleSpikelines", "togglehover", "togglespikelines", "toimage", "zoom", "zoom2d", "zoom3d", "zoomIn2d", "zoomInGeo", "zoomInMap", "zoomInMapbox", "zoomOut2d", "zoomOutGeo", "zoomOutMap", "zoomOutMapbox", "zoomin", "zoomout"]
v.main_fig.update_layout(modebar_remove=["toimage", "resetscale", "select", "lasso", "reset"])
v.main_fig.layout.dragmode = 'zoom'
if v.overview_fig:
    v.overview_fig.update_layout(modebar_remove=["toimage", "autoscale", "select", "lasso", "pan", "reset", "resetscale", "zoom", "zoomin", "zoomout"])
    v.overview_fig.layout.dragmode = False

display(container)